# Where is the next power outage?

**Name(s)**: Eric Sun & Sunan Xu

**Website Link**: https://ericsun153.github.io/Where_is_the_next_Outage/

# Code

In [1]:
import pandas as pd
import numpy as np
import os

import plotly.express as px
pd.options.plotting.backend = 'plotly'

import matplotlib as plt
import seaborn as sns
import openpyxl

## Framing the Problem

In [87]:
# Data Cleaning
xlsx_file = 'data/outage.xlsx'
csv_file = 'data/outage.csv'

df = pd.read_excel(xlsx_file)

# Write the DataFrame to a CSV file
df.to_csv(csv_file, index=False)
df = pd.read_csv('data/outage.csv')
df

,Major power outage events in the continental U.S.,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 47,Unnamed: 48,Unnamed: 49,Unnamed: 50,Unnamed: 51,Unnamed: 52,Unnamed: 53,Unnamed: 54,Unnamed: 55,Unnamed: 56
0,Time period: January 2000 - July 2016,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Regions affected: Outages reported in this dat...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,variables,OBS,YEAR,MONTH,U.S._STATE,POSTAL.CODE,NERC.REGION,CLIMATE.REGION,ANOMALY.LEVEL,CLIMATE.CATEGORY,...,POPPCT_URBAN,POPPCT_UC,POPDEN_URBAN,POPDEN_UC,POPDEN_RURAL,AREAPCT_URBAN,AREAPCT_UC,PCT_LAND,PCT_WATER_TOT,PCT_WATER_INLAND
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1535,NaN,1530,2011,12,North Dakota,ND,MRO,West North Central,-0.9,cold,...,59.9,19.9,2192.2,1868.2,3.9,0.27,0.1,97.5996492121418,2.40176525502843,2.40176525502843
1536,NaN,1531,2006,NaN,North Dakota,ND,MRO,West North Central,NaN,NaN,...,59.9,19.9,2192.2,1868.2,3.9,0.27,0.1,97.5996492121418,2.40176525502843,2.40176525502843
1537,NaN,1532,2009,8,South Dakota,SD,RFC,West North Central,0.5,warm,...,56.65,26.73,2038.3,1905.4,4.7,0.3,0.15,98.3077441776026,1.69225582239743,1.69225582239743
1538,NaN,1533,2009,8,South Dakota,SD,MRO,West North Central,0.5,warm,...,56.65,26.73,2038.3,1905.4,4.7,0.3,0.15,98.3077441776026,1.69225582239743,1.69225582239743


In [88]:
rows_to_skip = list(range(5))
df = pd.read_csv('data/outage.csv', skiprows=rows_to_skip, index_col='OBS')

# Combine the units line and column names, drop unecessary rows and columns
column = np.array(df.columns).astype('str')
unites = np.array(df.iloc[0].fillna('')).astype('str')
unites = ["(" + i + ")" for i in unites]
for i in range(len(unites)):
    if unites[i] == '()':
        unites[i] = ''
combined_column = np.core.defchararray.add(column, unites)
df.columns = combined_column
df = df.reset_index().drop(0).drop('variables(Units)', axis=1).reset_index(drop=True)
df

,OBS,YEAR,MONTH,U.S._STATE,POSTAL.CODE,NERC.REGION,CLIMATE.REGION,ANOMALY.LEVEL(numeric),CLIMATE.CATEGORY,"OUTAGE.START.DATE(Day of the week, Month Day, Year)",...,POPPCT_URBAN(%),POPPCT_UC(%),POPDEN_URBAN(persons per square mile),POPDEN_UC(persons per square mile),POPDEN_RURAL(persons per square mile),AREAPCT_URBAN(%),AREAPCT_UC(%),PCT_LAND(%),PCT_WATER_TOT(%),PCT_WATER_INLAND(%)
0,1.0,2011.0,7.0,Minnesota,MN,MRO,East North Central,-0.3,normal,2011-07-01 00:00:00,...,73.27,15.28,2279,1700.5,18.2,2.14,0.6,91.5926658691451,8.40733413085488,5.47874298334407
1,2.0,2014.0,5.0,Minnesota,MN,MRO,East North Central,-0.1,normal,2014-05-11 00:00:00,...,73.27,15.28,2279,1700.5,18.2,2.14,0.6,91.5926658691451,8.40733413085488,5.47874298334407
2,3.0,2010.0,10.0,Minnesota,MN,MRO,East North Central,-1.5,cold,2010-10-26 00:00:00,...,73.27,15.28,2279,1700.5,18.2,2.14,0.6,91.5926658691451,8.40733413085488,5.47874298334407
3,4.0,2012.0,6.0,Minnesota,MN,MRO,East North Central,-0.1,normal,2012-06-19 00:00:00,...,73.27,15.28,2279,1700.5,18.2,2.14,0.6,91.5926658691451,8.40733413085488,5.47874298334407
4,5.0,2015.0,7.0,Minnesota,MN,MRO,East North Central,1.2,warm,2015-07-18 00:00:00,...,73.27,15.28,2279,1700.5,18.2,2.14,0.6,91.5926658691451,8.40733413085488,5.47874298334407
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1529,1530.0,2011.0,12.0,North Dakota,ND,MRO,West North Central,-0.9,cold,2011-12-06 00:00:00,...,59.9,19.9,2192.2,1868.2,3.9,0.27,0.1,97.5996492121418,2.40176525502843,2.40176525502843
1530,1531.0,2006.0,NaN,North Dakota,ND,MRO,West North Central,NaN,NaN,NaN,...,59.9,19.9,2192.2,1868.2,3.9,0.27,0.1,97.5996492121418,2.40176525502843,2.40176525502843
1531,1532.0,2009.0,8.0,South Dakota,SD,RFC,West North Central,0.5,warm,2009-08-29 00:00:00,...,56.65,26.73,2038.3,1905.4,4.7,0.3,0.15,98.3077441776026,1.69225582239743,1.69225582239743
1532,1533.0,2009.0,8.0,South Dakota,SD,MRO,West North Central,0.5,warm,2009-08-29 00:00:00,...,56.65,26.73,2038.3,1905.4,4.7,0.3,0.15,98.3077441776026,1.69225582239743,1.69225582239743


In [89]:
df['YEAR'] = df['YEAR'].astype('int')
# Combine 'OUTAGE.START.DATE' and 'OUTAGE.START.TIME' into a new pd.Timestamp column called 'OUTAGE.START'
start_time = df['OUTAGE.START.TIME(Hour:Minute:Second (AM / PM))']
start_date = df['OUTAGE.START.DATE(Day of the week, Month Day, Year)']
df["OUTAGE.START"] = pd.to_datetime(start_date + " " + start_time)
# combine 'OUTAGE.RESTORATION.DATE' and 'OUTAGE.RESTORATION.TIME' into a new pd.Timestamp column called 'OUTAGE.RESTORATION'.
end_time = df['OUTAGE.RESTORATION.TIME(Hour:Minute:Second (AM / PM))']
end_date = df['OUTAGE.RESTORATION.DATE(Day of the week, Month Day, Year)']
df["OUTAGE.RESTORATION"] = pd.to_datetime(end_date + " " + end_time)
df[['OUTAGE.RESTORATION', 'OUTAGE.START']]

,OUTAGE.RESTORATION,OUTAGE.START
0,2011-07-03 20:00:00,2011-07-01 17:00:00
1,2014-05-11 18:39:00,2014-05-11 18:38:00
2,2010-10-28 22:00:00,2010-10-26 20:00:00
3,2012-06-20 23:00:00,2012-06-19 04:30:00
4,2015-07-19 07:00:00,2015-07-18 02:00:00
...,...,...
1529,2011-12-06 20:00:00,2011-12-06 08:00:00
1530,NaT,NaT
1531,2009-08-29 23:53:00,2009-08-29 22:54:00
1532,2009-08-29 14:01:00,2009-08-29 11:00:00


In [90]:
df['ANOMALY.LEVEL'] = df['ANOMALY.LEVEL(numeric)'].astype(float)
df = df.drop(columns=['ANOMALY.LEVEL(numeric)', 'OUTAGE.START.DATE(Day of the week, Month Day, Year)',
                      'OUTAGE.START.TIME(Hour:Minute:Second (AM / PM))', 
                      'OUTAGE.RESTORATION.DATE(Day of the week, Month Day, Year)',
                      'OUTAGE.RESTORATION.TIME(Hour:Minute:Second (AM / PM))'], axis=1)
columns_to_convert = {'OUTAGE.DURATION(mins)': float, 'DEMAND.LOSS.MW(Megawatt)': float,
                      'RES.PRICE(cents / kilowatt-hour)': float, 'COM.PRICE(cents / kilowatt-hour)': float,
                     'COM.PRICE(cents / kilowatt-hour)':float, 'IND.PRICE(cents / kilowatt-hour)': float,
                     'TOTAL.PRICE(cents / kilowatt-hour)':float, 'RES.SALES(Megawatt-hour)': float,
                     'COM.SALES(Megawatt-hour)':float, 'IND.SALES(Megawatt-hour)':float, 'TOTAL.PRICE(cents / kilowatt-hour)':float,
                     'RES.PERCEN(%)':float, 'COM.PERCEN(%)':float, 'IND.PERCEN(%)':float, 'RES.CUST.PCT(%)':float,
                     'COM.CUST.PCT(%)':float, 'IND.CUST.PCT(%)':float, 'PC.REALGSP.STATE(USD)':float, 'PC.REALGSP.USA(USD)':float,
                     'PC.REALGSP.REL(fraction)':float, 'PC.REALGSP.CHANGE(%)':float, 'UTIL.REALGSP(USD)':float, 'TOTAL.REALGSP(USD)':float,
                     'UTIL.CONTRI(%)':float, 'PI.UTIL.OFUSA(%)':float, 'POPPCT_URBAN(%)':float, 'POPPCT_UC(%)':float, 
                     'POPDEN_URBAN(persons per square mile)':float, 'POPDEN_UC(persons per square mile)':float, 
                     'POPDEN_RURAL(persons per square mile)':float, 'AREAPCT_URBAN(%)':float, 'AREAPCT_UC(%)':float,
                     'PCT_LAND(%)':float, 'PCT_WATER_TOT(%)':float, 'PCT_WATER_INLAND(%)':float,'TOTAL.SALES(Megawatt-hour)':float}
df = df.astype(columns_to_convert)
df

,OBS,YEAR,MONTH,U.S._STATE,POSTAL.CODE,NERC.REGION,CLIMATE.REGION,CLIMATE.CATEGORY,CAUSE.CATEGORY,CAUSE.CATEGORY.DETAIL,...,POPDEN_UC(persons per square mile),POPDEN_RURAL(persons per square mile),AREAPCT_URBAN(%),AREAPCT_UC(%),PCT_LAND(%),PCT_WATER_TOT(%),PCT_WATER_INLAND(%),OUTAGE.START,OUTAGE.RESTORATION,ANOMALY.LEVEL
0,1.0,2011,7.0,Minnesota,MN,MRO,East North Central,normal,severe weather,NaN,...,1700.5,18.2,2.14,0.60,91.592666,8.407334,5.478743,2011-07-01 17:00:00,2011-07-03 20:00:00,-0.3
1,2.0,2014,5.0,Minnesota,MN,MRO,East North Central,normal,intentional attack,vandalism,...,1700.5,18.2,2.14,0.60,91.592666,8.407334,5.478743,2014-05-11 18:38:00,2014-05-11 18:39:00,-0.1
2,3.0,2010,10.0,Minnesota,MN,MRO,East North Central,cold,severe weather,heavy wind,...,1700.5,18.2,2.14,0.60,91.592666,8.407334,5.478743,2010-10-26 20:00:00,2010-10-28 22:00:00,-1.5
3,4.0,2012,6.0,Minnesota,MN,MRO,East North Central,normal,severe weather,thunderstorm,...,1700.5,18.2,2.14,0.60,91.592666,8.407334,5.478743,2012-06-19 04:30:00,2012-06-20 23:00:00,-0.1
4,5.0,2015,7.0,Minnesota,MN,MRO,East North Central,warm,severe weather,NaN,...,1700.5,18.2,2.14,0.60,91.592666,8.407334,5.478743,2015-07-18 02:00:00,2015-07-19 07:00:00,1.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1529,1530.0,2011,12.0,North Dakota,ND,MRO,West North Central,cold,public appeal,NaN,...,1868.2,3.9,0.27,0.10,97.599649,2.401765,2.401765,2011-12-06 08:00:00,2011-12-06 20:00:00,-0.9
1530,1531.0,2006,NaN,North Dakota,ND,MRO,West North Central,NaN,fuel supply emergency,Coal,...,1868.2,3.9,0.27,0.10,97.599649,2.401765,2.401765,NaT,NaT,NaN
1531,1532.0,2009,8.0,South Dakota,SD,RFC,West North Central,warm,islanding,NaN,...,1905.4,4.7,0.30,0.15,98.307744,1.692256,1.692256,2009-08-29 22:54:00,2009-08-29 23:53:00,0.5
1532,1533.0,2009,8.0,South Dakota,SD,MRO,West North Central,warm,islanding,NaN,...,1905.4,4.7,0.30,0.15,98.307744,1.692256,1.692256,2009-08-29 11:00:00,2009-08-29 14:01:00,0.5


In [91]:
df = df.drop('OBS', axis=1)

In [92]:
df

,YEAR,MONTH,U.S._STATE,POSTAL.CODE,NERC.REGION,CLIMATE.REGION,CLIMATE.CATEGORY,CAUSE.CATEGORY,CAUSE.CATEGORY.DETAIL,HURRICANE.NAMES,...,POPDEN_UC(persons per square mile),POPDEN_RURAL(persons per square mile),AREAPCT_URBAN(%),AREAPCT_UC(%),PCT_LAND(%),PCT_WATER_TOT(%),PCT_WATER_INLAND(%),OUTAGE.START,OUTAGE.RESTORATION,ANOMALY.LEVEL
0,2011,7.0,Minnesota,MN,MRO,East North Central,normal,severe weather,NaN,NaN,...,1700.5,18.2,2.14,0.60,91.592666,8.407334,5.478743,2011-07-01 17:00:00,2011-07-03 20:00:00,-0.3
1,2014,5.0,Minnesota,MN,MRO,East North Central,normal,intentional attack,vandalism,NaN,...,1700.5,18.2,2.14,0.60,91.592666,8.407334,5.478743,2014-05-11 18:38:00,2014-05-11 18:39:00,-0.1
2,2010,10.0,Minnesota,MN,MRO,East North Central,cold,severe weather,heavy wind,NaN,...,1700.5,18.2,2.14,0.60,91.592666,8.407334,5.478743,2010-10-26 20:00:00,2010-10-28 22:00:00,-1.5
3,2012,6.0,Minnesota,MN,MRO,East North Central,normal,severe weather,thunderstorm,NaN,...,1700.5,18.2,2.14,0.60,91.592666,8.407334,5.478743,2012-06-19 04:30:00,2012-06-20 23:00:00,-0.1
4,2015,7.0,Minnesota,MN,MRO,East North Central,warm,severe weather,NaN,NaN,...,1700.5,18.2,2.14,0.60,91.592666,8.407334,5.478743,2015-07-18 02:00:00,2015-07-19 07:00:00,1.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1529,2011,12.0,North Dakota,ND,MRO,West North Central,cold,public appeal,NaN,NaN,...,1868.2,3.9,0.27,0.10,97.599649,2.401765,2.401765,2011-12-06 08:00:00,2011-12-06 20:00:00,-0.9
1530,2006,NaN,North Dakota,ND,MRO,West North Central,NaN,fuel supply emergency,Coal,NaN,...,1868.2,3.9,0.27,0.10,97.599649,2.401765,2.401765,NaT,NaT,NaN
1531,2009,8.0,South Dakota,SD,RFC,West North Central,warm,islanding,NaN,NaN,...,1905.4,4.7,0.30,0.15,98.307744,1.692256,1.692256,2009-08-29 22:54:00,2009-08-29 23:53:00,0.5
1532,2009,8.0,South Dakota,SD,MRO,West North Central,warm,islanding,NaN,NaN,...,1905.4,4.7,0.30,0.15,98.307744,1.692256,1.692256,2009-08-29 11:00:00,2009-08-29 14:01:00,0.5


We will predict the state (or the place) that happens power outage after 2012 (since 2013). As a aresult, we will drop the timestamp and any columns that directly equalize to the postal code column (eg. U.S.\_state). We need to clear out the columns that are the same if the state is the same, and keep the columns that are different among each outage instead of the state characteristics.

In [93]:
df = df.drop(columns=['OUTAGE.START', 'OUTAGE.RESTORATION', 'U.S._STATE', 'HURRICANE.NAMES', 
                      'POPDEN_URBAN(persons per square mile)', 'POPPCT_URBAN(%)', 'POPPCT_UC(%)',
                     'POPDEN_URBAN(persons per square mile)', 'POPDEN_UC(persons per square mile)',
                     'POPDEN_RURAL(persons per square mile)', 'AREAPCT_URBAN(%)', 'PCT_LAND(%)', 'PCT_WATER_TOT(%)',
                     'PCT_WATER_INLAND(%)', 'AREAPCT_UC(%)'])
df

,YEAR,MONTH,POSTAL.CODE,NERC.REGION,CLIMATE.REGION,CLIMATE.CATEGORY,CAUSE.CATEGORY,CAUSE.CATEGORY.DETAIL,OUTAGE.DURATION(mins),DEMAND.LOSS.MW(Megawatt),...,PC.REALGSP.STATE(USD),PC.REALGSP.USA(USD),PC.REALGSP.REL(fraction),PC.REALGSP.CHANGE(%),UTIL.REALGSP(USD),TOTAL.REALGSP(USD),UTIL.CONTRI(%),PI.UTIL.OFUSA(%),POPULATION,ANOMALY.LEVEL
0,2011,7.0,MN,MRO,East North Central,normal,severe weather,NaN,3060.0,NaN,...,51268.0,47586.0,1.077376,1.6,4802.0,274182.0,1.751391,2.2,5348119.0,-0.3
1,2014,5.0,MN,MRO,East North Central,normal,intentional attack,vandalism,1.0,NaN,...,53499.0,49091.0,1.089792,1.9,5226.0,291955.0,1.790002,2.2,5457125.0,-0.1
2,2010,10.0,MN,MRO,East North Central,cold,severe weather,heavy wind,3000.0,NaN,...,50447.0,47287.0,1.066826,2.7,4571.0,267895.0,1.706266,2.1,5310903.0,-1.5
3,2012,6.0,MN,MRO,East North Central,normal,severe weather,thunderstorm,2550.0,NaN,...,51598.0,48156.0,1.071476,0.6,5364.0,277627.0,1.932089,2.2,5380443.0,-0.1
4,2015,7.0,MN,MRO,East North Central,warm,severe weather,NaN,1740.0,250.0,...,54431.0,49844.0,1.092027,1.7,4873.0,292023.0,1.668704,2.2,5489594.0,1.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1529,2011,12.0,ND,MRO,West North Central,cold,public appeal,NaN,720.0,155.0,...,57012.0,47586.0,1.198083,9.8,934.0,39067.0,2.390765,0.5,685326.0,-0.9
1530,2006,NaN,ND,MRO,West North Central,NaN,fuel supply emergency,Coal,NaN,1650.0,...,42913.0,48909.0,0.877405,3.5,1019.0,27868.0,3.656524,0.7,649422.0,NaN
1531,2009,8.0,SD,RFC,West North Central,warm,islanding,NaN,59.0,84.0,...,45230.0,46680.0,0.968937,0.0,606.0,36504.0,1.660092,0.3,807067.0,0.5
1532,2009,8.0,SD,MRO,West North Central,warm,islanding,NaN,181.0,373.0,...,45230.0,46680.0,0.968937,0.0,606.0,36504.0,1.660092,0.3,807067.0,0.5


In [94]:
df.isnull().sum()

YEAR                                    0
MONTH                                   9
POSTAL.CODE                             0
NERC.REGION                             0
CLIMATE.REGION                          6
CLIMATE.CATEGORY                        9
CAUSE.CATEGORY                          0
CAUSE.CATEGORY.DETAIL                 471
OUTAGE.DURATION(mins)                  58
DEMAND.LOSS.MW(Megawatt)              705
CUSTOMERS.AFFECTED                    443
RES.PRICE(cents / kilowatt-hour)       22
COM.PRICE(cents / kilowatt-hour)       22
IND.PRICE(cents / kilowatt-hour)       22
TOTAL.PRICE(cents / kilowatt-hour)     22
RES.SALES(Megawatt-hour)               22
COM.SALES(Megawatt-hour)               22
IND.SALES(Megawatt-hour)               22
TOTAL.SALES(Megawatt-hour)             22
RES.PERCEN(%)                          22
COM.PERCEN(%)                          22
IND.PERCEN(%)                          22
RES.CUSTOMERS                           0
COM.CUSTOMERS                     

In [95]:
df[['POSTAL.CODE', 'TOTAL.CUSTOMERS']]

,POSTAL.CODE,TOTAL.CUSTOMERS
0,MN,2595696.0
1,MN,2640737.0
2,MN,2586905.0
3,MN,2606813.0
4,MN,2673531.0
...,...,...
1529,ND,394394.0
1530,ND,366037.0
1531,SD,436229.0
1532,SD,436229.0


Should impute the total duration using the average outage duration for each state.

In [105]:
mean_duration_by_state = df.groupby('POSTAL.CODE')['OUTAGE.DURATION(mins)'].mean()
mean_duration_by_state['AK'] = 1860
df['OUTAGE.DURATION(mins)'].fillna(df['POSTAL.CODE'].map(mean_duration_by_state), inplace=True)
df

,YEAR,MONTH,POSTAL.CODE,NERC.REGION,CLIMATE.REGION,CLIMATE.CATEGORY,CAUSE.CATEGORY,CAUSE.CATEGORY.DETAIL,OUTAGE.DURATION(mins),DEMAND.LOSS.MW(Megawatt),...,PC.REALGSP.STATE(USD),PC.REALGSP.USA(USD),PC.REALGSP.REL(fraction),PC.REALGSP.CHANGE(%),UTIL.REALGSP(USD),TOTAL.REALGSP(USD),UTIL.CONTRI(%),PI.UTIL.OFUSA(%),POPULATION,ANOMALY.LEVEL
0,2011,7.0,MN,MRO,East North Central,normal,severe weather,NaN,3060.0,NaN,...,51268.0,47586.0,1.077376,1.6,4802.0,274182.0,1.751391,2.2,5348119.0,-0.3
1,2014,5.0,MN,MRO,East North Central,normal,intentional attack,vandalism,1.0,NaN,...,53499.0,49091.0,1.089792,1.9,5226.0,291955.0,1.790002,2.2,5457125.0,-0.1
2,2010,10.0,MN,MRO,East North Central,cold,severe weather,heavy wind,3000.0,NaN,...,50447.0,47287.0,1.066826,2.7,4571.0,267895.0,1.706266,2.1,5310903.0,-1.5
3,2012,6.0,MN,MRO,East North Central,normal,severe weather,thunderstorm,2550.0,NaN,...,51598.0,48156.0,1.071476,0.6,5364.0,277627.0,1.932089,2.2,5380443.0,-0.1
4,2015,7.0,MN,MRO,East North Central,warm,severe weather,NaN,1740.0,250.0,...,54431.0,49844.0,1.092027,1.7,4873.0,292023.0,1.668704,2.2,5489594.0,1.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1529,2011,12.0,ND,MRO,West North Central,cold,public appeal,NaN,720.0,155.0,...,57012.0,47586.0,1.198083,9.8,934.0,39067.0,2.390765,0.5,685326.0,-0.9
1530,2006,NaN,ND,MRO,West North Central,NaN,fuel supply emergency,Coal,720.0,1650.0,...,42913.0,48909.0,0.877405,3.5,1019.0,27868.0,3.656524,0.7,649422.0,NaN
1531,2009,8.0,SD,RFC,West North Central,warm,islanding,NaN,59.0,84.0,...,45230.0,46680.0,0.968937,0.0,606.0,36504.0,1.660092,0.3,807067.0,0.5
1532,2009,8.0,SD,MRO,West North Central,warm,islanding,NaN,181.0,373.0,...,45230.0,46680.0,0.968937,0.0,606.0,36504.0,1.660092,0.3,807067.0,0.5


In [110]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1534 entries, 0 to 1533
Data columns (total 39 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   YEAR                                1534 non-null   int32  
 1   MONTH                               1525 non-null   float64
 2   POSTAL.CODE                         1534 non-null   object 
 3   NERC.REGION                         1534 non-null   object 
 4   CLIMATE.REGION                      1528 non-null   object 
 5   CLIMATE.CATEGORY                    1525 non-null   object 
 6   CAUSE.CATEGORY                      1534 non-null   object 
 7   CAUSE.CATEGORY.DETAIL               1063 non-null   object 
 8   OUTAGE.DURATION(mins)               1534 non-null   float64
 9   DEMAND.LOSS.MW(Megawatt)            829 non-null    float64
 10  CUSTOMERS.AFFECTED                  1091 non-null   float64
 11  RES.PRICE(cents / kilowatt-hour)    1512 no

In [106]:
train_set = df[df['YEAR'] <= 2012]
test_set = df.drop(train_set.index)  # Exclude the training set rows to get the test set

In [137]:
train_set

,YEAR,MONTH,POSTAL.CODE,NERC.REGION,CLIMATE.REGION,CLIMATE.CATEGORY,CAUSE.CATEGORY,CAUSE.CATEGORY.DETAIL,OUTAGE.DURATION(mins),DEMAND.LOSS.MW(Megawatt),...,PC.REALGSP.STATE(USD),PC.REALGSP.USA(USD),PC.REALGSP.REL(fraction),PC.REALGSP.CHANGE(%),UTIL.REALGSP(USD),TOTAL.REALGSP(USD),UTIL.CONTRI(%),PI.UTIL.OFUSA(%),POPULATION,ANOMALY.LEVEL
0,2011,7.0,MN,MRO,East North Central,normal,severe weather,NaN,3060.0,NaN,...,51268.0,47586.0,1.077376,1.6,4802.0,274182.0,1.751391,2.2,5348119.0,-0.3
2,2010,10.0,MN,MRO,East North Central,cold,severe weather,heavy wind,3000.0,NaN,...,50447.0,47287.0,1.066826,2.7,4571.0,267895.0,1.706266,2.1,5310903.0,-1.5
3,2012,6.0,MN,MRO,East North Central,normal,severe weather,thunderstorm,2550.0,NaN,...,51598.0,48156.0,1.071476,0.6,5364.0,277627.0,1.932089,2.2,5380443.0,-0.1
5,2010,11.0,MN,MRO,East North Central,cold,severe weather,winter storm,1860.0,NaN,...,50447.0,47287.0,1.066826,2.7,4571.0,267895.0,1.706266,2.1,5310903.0,-1.4
6,2010,7.0,MN,MRO,East North Central,cold,severe weather,tornadoes,2970.0,NaN,...,50447.0,47287.0,1.066826,2.7,4571.0,267895.0,1.706266,2.1,5310903.0,-0.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1529,2011,12.0,ND,MRO,West North Central,cold,public appeal,NaN,720.0,155.0,...,57012.0,47586.0,1.198083,9.8,934.0,39067.0,2.390765,0.5,685326.0,-0.9
1530,2006,NaN,ND,MRO,West North Central,NaN,fuel supply emergency,Coal,720.0,1650.0,...,42913.0,48909.0,0.877405,3.5,1019.0,27868.0,3.656524,0.7,649422.0,NaN
1531,2009,8.0,SD,RFC,West North Central,warm,islanding,NaN,59.0,84.0,...,45230.0,46680.0,0.968937,0.0,606.0,36504.0,1.660092,0.3,807067.0,0.5
1532,2009,8.0,SD,MRO,West North Central,warm,islanding,NaN,181.0,373.0,...,45230.0,46680.0,0.968937,0.0,606.0,36504.0,1.660092,0.3,807067.0,0.5


In [138]:
test_set

,YEAR,MONTH,POSTAL.CODE,NERC.REGION,CLIMATE.REGION,CLIMATE.CATEGORY,CAUSE.CATEGORY,CAUSE.CATEGORY.DETAIL,OUTAGE.DURATION(mins),DEMAND.LOSS.MW(Megawatt),...,PC.REALGSP.STATE(USD),PC.REALGSP.USA(USD),PC.REALGSP.REL(fraction),PC.REALGSP.CHANGE(%),UTIL.REALGSP(USD),TOTAL.REALGSP(USD),UTIL.CONTRI(%),PI.UTIL.OFUSA(%),POPULATION,ANOMALY.LEVEL
1,2014,5.0,MN,MRO,East North Central,normal,intentional attack,vandalism,1.000000,NaN,...,53499.0,49091.0,1.089792,1.9,5226.0,291955.0,1.790002,2.2,5457125.0,-0.1
4,2015,7.0,MN,MRO,East North Central,warm,severe weather,NaN,1740.000000,250.0,...,54431.0,49844.0,1.092027,1.7,4873.0,292023.0,1.668704,2.2,5489594.0,1.2
8,2015,3.0,MN,MRO,East North Central,warm,intentional attack,sabotage,155.000000,20.0,...,54431.0,49844.0,1.092027,1.7,4873.0,292023.0,1.668704,2.2,5489594.0,0.6
9,2013,6.0,MN,MRO,East North Central,normal,severe weather,hailstorm,3621.000000,NaN,...,52479.0,48396.0,1.084366,1.7,5613.0,284542.0,1.972644,2.3,5420541.0,-0.2
10,2013,6.0,MN,MRO,East North Central,normal,severe weather,hailstorm,7740.000000,NaN,...,52479.0,48396.0,1.084366,1.7,5613.0,284542.0,1.972644,2.3,5420541.0,-0.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1513,2015,6.0,KS,SPP,South,warm,severe weather,NaN,4376.285714,NaN,...,45558.0,49844.0,0.914012,0.0,2290.0,135967.0,1.684232,1.1,2911641.0,1.0
1520,2016,7.0,ID,WECC,Northwest,normal,system operability disruption,uncontrolled loss,224.000000,290.0,...,36256.0,50660.0,0.715673,2.7,960.0,60911.0,1.576070,0.4,1680026.0,-0.3
1526,2016,3.0,ID,WECC,Northwest,warm,intentional attack,sabotage,0.000000,0.0,...,36256.0,50660.0,0.715673,2.7,960.0,60911.0,1.576070,0.4,1680026.0,1.6
1527,2016,3.0,ID,WECC,Northwest,warm,intentional attack,vandalism,414.625000,0.0,...,36256.0,50660.0,0.715673,2.7,960.0,60911.0,1.576070,0.4,1680026.0,1.6


## Baseline Model

In [118]:
# Model Setup
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression

Use three features: `NERC.REGION`, `CAUSE.CATEGORY`, and `OUTAGE.DURATION(mins)`

In [126]:
X_train = train_set[['NERC.REGION', 'CAUSE.CATEGORY', 'OUTAGE.DURATION(mins)']]
y_train = train_set['POSTAL.CODE']
X_test = test_set[['NERC.REGION', 'CAUSE.CATEGORY', 'OUTAGE.DURATION(mins)']]
y_test = test_set['POSTAL.CODE']
pipeline = Pipeline([
    ('preprocessing', ColumnTransformer([
        ('onehot', OneHotEncoder(), ['NERC.REGION', 'CAUSE.CATEGORY']),
        ('scaling', StandardScaler(), ['OUTAGE.DURATION(mins)'])
    ], remainder='passthrough')),
    ('model', LogisticRegression(max_iter=1000))
])

pipeline.fit(X_train, y_train)
predictions = pipeline.predict(X_train)
accuracy_train = pipeline.score(X_train, y_train)
print("Training Accuracy:", accuracy_train)
print("Testing Accuracy:", pipeline.score(X_test, y_test))

Training Accuracy: 0.5114573785517873
Testing Accuracy: 0.37697516930022573


In [136]:
train_set['POSTAL.CODE'].value_counts(normalize=True)

CA    0.142988
TX    0.080660
MI    0.068744
WA    0.064161
NY    0.048579
PA    0.042163
MD    0.040330
FL    0.037580
IL    0.036664
OH    0.034830
LA    0.032997
IN    0.030247
VA    0.028414
NJ    0.024748
NC    0.024748
DE    0.021082
AR    0.015582
UT    0.014665
ME    0.014665
OK    0.013749
CT    0.012832
GA    0.012832
TN    0.012832
MA    0.011916
NH    0.011916
AZ    0.010999
CO    0.010082
KY    0.010082
OR    0.009166
MN    0.008249
MO    0.008249
DC    0.006416
VT    0.006416
WI    0.005500
SC    0.004583
HI    0.004583
ID    0.004583
NV    0.003666
WY    0.003666
KS    0.003666
NE    0.003666
IA    0.003666
AL    0.002750
NM    0.001833
ND    0.001833
SD    0.001833
WV    0.001833
MT    0.000917
MS    0.000917
AK    0.000917
Name: POSTAL.CODE, dtype: float64

In [139]:
test_set['POSTAL.CODE'].value_counts(normalize=True)

CA    0.121896
TX    0.088036
WA    0.060948
UT    0.056433
TN    0.045147
MI    0.045147
DE    0.040632
NY    0.040632
AZ    0.036117
OR    0.036117
MD    0.031603
WI    0.031603
NC    0.029345
PA    0.024831
IN    0.022573
OK    0.020316
AR    0.018059
NJ    0.018059
MO    0.018059
VA    0.013544
NM    0.013544
MN    0.013544
IL    0.013544
KS    0.011287
OH    0.011287
MA    0.011287
ID    0.009029
IA    0.009029
LA    0.009029
FL    0.009029
CO    0.009029
CT    0.009029
GA    0.006772
NV    0.006772
ME    0.006772
DC    0.006772
SC    0.006772
MS    0.006772
AL    0.006772
VT    0.004515
WV    0.004515
WY    0.004515
KY    0.004515
MT    0.004515
NH    0.002257
Name: POSTAL.CODE, dtype: float64

## Final Model

In [3]:
# TODO

## Fairness Analysis

In [ ]:
# TODO